In [2]:
import lightning as L
from E_coefficients_model import NirbModule
import numpy as np
import torch
import random 
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
from helpers import load_pint_data, inverse_min_max_scaler
import pyvista as pv
import pickle

In [3]:
import sys
sys.path.append(str(Path().cwd().parent.parent))
from ComsolClasses.comsol_classes import COMSOL_VTU
from ComsolClasses.helper import calculate_normal

In [4]:
VERSION = "01"
ROOT = Path.cwd().parent / "Snapshots" / VERSION
ROOT.exists()

True

In [5]:
chk_pt_paths = [path for path in (ROOT / "nn_logs").rglob("*.ckpt")]
chk_pt_paths = sorted(chk_pt_paths, key = lambda p: p.parent.parent.stem)

for idx, path in enumerate(chk_pt_paths):
    print(f"{idx:02d} ({path.parent.parent.stem}): {path.name}")

00 (version_14): epoch=19999-step=60000.ckpt
01 (version_15): epoch=19999-step=60000.ckpt
02 (version_16): epoch=39999-step=120000.ckpt
03 (version_20): epoch=99999-step=300000.ckpt
04 (version_22): epoch=63962-step=191889.ckpt
05 (version_23): epoch=41471-step=82944.ckpt
06 (version_24): epoch=199999-step=400000.ckpt
07 (version_25): epoch=299999-step=600000.ckpt
08 (version_26): epoch=52520-step=105042.ckpt
09 (version_27): epoch=99999-step=200000.ckpt
10 (version_28): epoch=249999-step=500000.ckpt
11 (version_30): epoch=236356-step=236357.ckpt
12 (version_33): epoch=7174-step=7175.ckpt
13 (version_34): epoch=59529-step=59530.ckpt
14 (version_35): epoch=199999-step=200000.ckpt
15 (version_36): epoch=160489-step=160490.ckpt


In [6]:
# ckpt = torch.load(chk_pt_paths[10], map_location="cpu")
# for key in ckpt['state_dict'].keys():
#     print(key)
# for k, v in ckpt['state_dict'].items():
#     print(k, v.shape)
# print(ckpt.get("hyper_parameters", {}))
# print(ckpt.get("hyper_parameters", {}))

In [7]:
chk_pt_path = chk_pt_paths[11]

In [8]:

trained_model = NirbModule.load_from_checkpoint(chk_pt_path)
trained_model = trained_model.to('cpu')
trained_model.eval()

/Users/thomassimader/Documents/NIRB/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'activation' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['activation'])`.


NirbModule(
  (loss): MSELoss()
  (activation): Sigmoid()
  (model): NIRB_NN(
    (layers): Sequential(
      (0): Linear(in_features=2, out_features=6, bias=True)
      (1): Sigmoid()
      (2): Linear(in_features=6, out_features=16, bias=True)
      (3): Sigmoid()
      (4): Linear(in_features=16, out_features=32, bias=True)
      (5): Sigmoid()
      (6): Linear(in_features=32, out_features=64, bias=True)
      (7): Sigmoid()
      (8): Linear(in_features=64, out_features=32, bias=True)
      (9): Sigmoid()
      (10): Linear(in_features=32, out_features=7, bias=True)
    )
  )
  (msa_metric): MeanAbsoluteError()
)

In [9]:
def mse(predictions, targets):
    return np.mean(np.square(predictions - targets))

In [10]:
basis_functions             = np.load(ROOT / "BasisFunctions" / "basis_fts_matrix.npy")
training_snapshots_npy      = np.load(ROOT / "Exports" / "temperatures.npy")
training_parameters_npy     = load_pint_data(ROOT / "training_samples.csv", is_numpy = True)
min_max                     = np.load(ROOT / "BasisFunctions" / "min_max.npy")


# test_snapshots            = np.load('Data/GrSbk_val_snapshots.npy')[0, :, :]
# test_parameters       = np.load('Data/GrSbk_ValidationParameters.npy')


In [11]:
# Prepare data
n_training = 80
training_snapshots  = training_snapshots_npy[:n_training, -1, :] # last time step
training_parameters = training_parameters_npy[:n_training, 2:]
test_snapshots      = training_snapshots_npy[n_training:, -1, :]
test_parameters     = training_parameters_npy[n_training:, 2:]

In [12]:
training_vtus = list((ROOT / "Training").glob("Training_*"))
test_snapshots = sorted([p for p in training_vtus if int(p.stem.split("_")[1]) >= n_training])


In [13]:
# Calculate mean/var from training snapshots
mean = np.mean(training_parameters, axis=0)
var = np.var(training_parameters, axis=0)
# Standardization
training_parameters    = (training_parameters - mean)/np.sqrt(var)
test_parameters        = (test_parameters - mean)/np.sqrt(var)

In [14]:
predictions_train = trained_model(torch.from_numpy(training_parameters.astype(np.float32))) 
predictions_test  = trained_model(torch.from_numpy(test_parameters.astype(np.float32)))

In [15]:
full_solution_train = np.matmul(predictions_train.detach().numpy(), basis_functions)
full_solution_test = np.matmul(predictions_test.detach().numpy(), basis_functions)

In [16]:
mse_train = mse(full_solution_train, training_snapshots)
# mse_test = mse(full_solution_test, test_snapshots )

In [17]:
# Variance of targets
# var_val = np.var(test_snapshots, axis=0)
var_train = np.var(training_snapshots, axis=0)

# R² and Q²
R2 = 1 - (np.sum((full_solution_train-training_snapshots)**2) / np.sum((training_snapshots-np.mean(training_snapshots))**2))
# Q2 =  1 - (np.sum((full_solution_test-test_snapshots)**2) / np.sum((test_snapshots-np.mean(test_snapshots))**2))

# Aggregated if needed
toShowR2 = np.average(R2)
# toReturnQ2 = np.average(Q2)
print(toShowR2)
# print(toReturnQ2)
print(f"{np.average(mse_train):.3e}")
# print(f"{np.average(mse_test):.3e}")

-66.97588766425872
1.383e+05


In [18]:
comsol_data = COMSOL_VTU("/Users/thomassimader/Documents/NIRB/Snapshots/01/Training/Training_000.vtu")
normal = calculate_normal(60, 90)

In [ ]:
N = 5
random.seed(3123)
samples = random.sample(range(len(test_snapshots)), N)
colors = px.colors.sample_colorscale("thermal", [n/(N -1) for n in range(N)])
# Initialize the figure
n_cols = 2  # number of columns you want

plotter = pv.Plotter(shape=(N, n_cols), window_size=(1300, 1200))
counter = 0
# Loop through the samples and plot
for i in range(N):
    sample_idx = samples[i]
    plotter.subplot(i, 0)
    counter += 1
    field_name = f"Test Sample {sample_idx}"
    
    param = test_parameters[sample_idx]
    param_t = torch.from_numpy(param.astype(np.float32)).view(1, -1)
    res = trained_model(param_t)
    res_np = res.detach().numpy()
    my_sol = np.dot(res_np.flatten(), basis_functions)
    my_sol = inverse_min_max_scaler(my_sol, min_max[0], min_max[1])
    
    comsol_data.mesh.point_data[field_name] = my_sol
    clipped = comsol_data.mesh.clip(normal=-np.array(normal), origin = comsol_data.mesh.center)
    
    plotter.add_mesh(clipped, scalars=field_name,
                        cmap='jet',
                        scalar_bar_args={'title': f'Temperature [K] ({counter})',
                                        'label_font_size': 10,
                                        'title_font_size': 8,})
    plotter.add_text(f"NIRB (Sample {sample_idx:03d})")
    plotter.add_axes(line_width=1.)
    plotter.add_bounding_box()
    plotter.show_grid(
        font_size=6,
        # location='outer',
        n_xlabels=3,  # number of labels (ticks) on x-axis
        n_ylabels=3,  # number of labels (ticks) on y-axis
        n_zlabels=3,  # number of labels (ticks) on z-axis
        color='gray',
        xtitle='',
        ytitle='',
        ztitle='',
                        )
    plotter.subplot(i, 1)
    counter += 1
    test_snap = COMSOL_VTU(test_snapshots[sample_idx])
    clipped = test_snap.mesh.clip(normal=-np.array(normal), origin = comsol_data.mesh.center)
    plotter.add_mesh(clipped, scalars=test_snap.format_fied("Temperature", -1),
                    cmap='jet',
                    scalar_bar_args={'title': f'Temperature [K] ({counter})',
                                    'label_font_size': 10,
                                    'title_font_size': 8,})
    plotter.add_text(f"Original (Sample {sample_idx:03d})")
    plotter.add_axes(line_width=1.)
    plotter.add_bounding_box()
    plotter.show_grid(
        font_size=6,
        # location='outer',
        n_xlabels=3,  # number of labels (ticks) on x-axis
        n_ylabels=3,  # number of labels (ticks) on y-axis
        n_zlabels=3,  # number of labels (ticks) on z-axis
        color='gray',
        xtitle='',
        ytitle='',
        ztitle='',
                        )
    
    

plotter.show()


Widget(value='<iframe src="http://localhost:52115/index.html?ui=P_0x1646c3b30_1&reconnect=auto" class="pyvista…